In [1]:
# --------------------------------------------------------  
# Faster R-CNN  
# Copyright (c) 2015 Microsoft  
# Licensed under The MIT License [see LICENSE for details]  
# Written by Ross Girshick and Sean Bell  
# --------------------------------------------------------  
# caffe_root ="/home/lqp/py-faster-rcnn/caffe-fast-rcnn/"
# import sys
# sys.path.insert(0,caffe_root+"python")
import os  
import caffe  
import yaml  
from fast_rcnn.config import cfg  
import numpy as np  
import numpy.random as npr  
from generate_anchors import generate_anchors  
from utils.cython_bbox import bbox_overlaps  
from fast_rcnn.bbox_transform import bbox_transform  
  
DEBUG = False  
  
class AnchorTargetLayer(caffe.Layer):  
    """ 
    Assign anchors to ground-truth targets. Produces anchor classification 
    labels and bounding-box regression targets. 
    """  
  
    def setup(self, bottom, top):  
        layer_params = yaml.load(self.param_str_)  
        anchor_scales = layer_params.get('scales', (8, 16, 32))  
        self._anchors = generate_anchors(scales=np.array(anchor_scales))#九个anchor的w h x_cstr y_cstr，对原始的wh做横向纵向变化，并放大缩小得到九个  
        self._num_anchors = self._anchors.shape[0]#anchor的个数</span>  
        self._feat_stride = layer_params['feat_stride']#网络中参数16  
  
        if DEBUG:  
            print 'anchors:'  
            print self._anchors  
            print 'anchor shapes:'  
            print np.hstack((  
                self._anchors[:, 2::4] - self._anchors[:, 0::4],  
                self._anchors[:, 3::4] - self._anchors[:, 1::4],  
            ))  
            self._counts = cfg.EPS  
            self._sums = np.zeros((1, 4))  
            self._squared_sums = np.zeros((1, 4))  
            self._fg_sum = 0  
            self._bg_sum = 0  
            self._count = 0  
  
        # allow boxes to sit over the edge by a small amount  
        self._allowed_border = layer_params.get('allowed_border', 0)  
        #bottom 长度为4；bottom[0],map；bottom[1],boxes,labels;bottom[2],im_fo;bottom[3],图片数据  
        height, width = bottom[0].data.shape[-2:]  
        if DEBUG:  
            print 'AnchorTargetLayer: height', height, 'width', width  
  
        A = self._num_anchors#anchor的个数  
        # labels  
        top[0].reshape(1, 1, A * height, width)  
        # bbox_targets  
        top[1].reshape(1, A * 4, height, width)  
        # bbox_inside_weights  
        top[2].reshape(1, A * 4, height, width)  
        # bbox_outside_weights  
        top[3].reshape(1, A * 4, height, width)  
  
    def forward(self, bottom, top):  
        # Algorithm:  
        #  
        # for each (H, W) location i  
        #   generate 9 anchor boxes centered on cell i  
        #   apply predicted bbox deltas at cell i to each of the 9 anchors  
        # filter out-of-image anchors  
        # measure GT overlap  
  
        assert bottom[0].data.shape[0] == 1,\
            'Only single item batches are supported'  
  
        # map of shape (..., H, W)  
        height, width = bottom[0].data.shape[-2:]  
        # GT boxes (x1, y1, x2, y2, label)  
        gt_boxes = bottom[1].data#gt_boxes:长度不定  
        # im_info  
        im_info = bottom[2].data[0, :]  
  
        if DEBUG:  
            print ''  
            print 'im_size: ({}, {})'.format(im_info[0], im_info[1])  
            print 'scale: {}'.format(im_info[2])  
            print 'height, width: ({}, {})'.format(height, width)  
            print 'rpn: gt_boxes.shape', gt_boxes.shape  
            print 'rpn: gt_boxes', gt_boxes  
  
        # 1. Generate proposals from bbox deltas and shifted anchors  
        shift_x = np.arange(0, width) * self._feat_stride  
        shift_y = np.arange(0, height) * self._feat_stride  
        shift_x, shift_y = np.meshgrid(shift_x, shift_y)  
        shifts = np.vstack((shift_x.ravel(), shift_y.ravel(),  
                            shift_x.ravel(), shift_y.ravel())).transpose()  
        # add A anchors (1, A, 4) to  
        # cell K shifts (K, 1, 4) to get  
        # shift anchors (K, A, 4)  
        # reshape to (K*A, 4) shifted anchors  
        A = self._num_anchors  
        K = shifts.shape[0]  
        all_anchors = (self._anchors.reshape((1, A, 4)) +  
                       shifts.reshape((1, K, 4)).transpose((1, 0, 2)))  
        all_anchors = all_anchors.reshape((K * A, 4))  
        total_anchors = int(K * A)#K*A，所有anchors个数，包括越界的  
        #K: width*height  
        #A: 9  
        # only keep anchors inside the image  
        inds_inside = np.where(  
            (all_anchors[:, 0] >= -self._allowed_border) &  
            (all_anchors[:, 1] >= -self._allowed_border) &  
            (all_anchors[:, 2] < im_info[1] + self._allowed_border) &  # width  
            (all_anchors[:, 3] < im_info[0] + self._allowed_border)    # height  
        )[0]#没有过界的anchors索引  
  
        if DEBUG:  
            print 'total_anchors', total_anchors  
            print 'inds_inside', len(inds_inside)  
  
        # keep only inside anchors  
        anchors = all_anchors[inds_inside, :]#没有过界的anchors  
        if DEBUG:  
            print 'anchors.shape', anchors.shape  
  
        # label: 1 is positive, 0 is negative, -1 is dont care  
        labels = np.empty((len(inds_inside), ), dtype=np.float32)  
        labels.fill(-1)  
  
        # overlaps between the anchors and the gt boxes  
        # overlaps (ex, gt)  
        overlaps = bbox_overlaps(  
            np.ascontiguousarray(anchors, dtype=np.float),  
            np.ascontiguousarray(gt_boxes, dtype=np.float))  
        argmax_overlaps = overlaps.argmax(axis=1)#overlaps每行最大值索引  
        max_overlaps = overlaps[np.arange(len(inds_inside)), argmax_overlaps]  
        gt_argmax_overlaps = overlaps.argmax(axis=0)  
        gt_max_overlaps = overlaps[gt_argmax_overlaps,  
                                   np.arange(overlaps.shape[1])]  
        gt_argmax_overlaps = np.where(overlaps == gt_max_overlaps)[0]  
  
        if not cfg.TRAIN.RPN_CLOBBER_POSITIVES:  
            # assign bg labels first so that positive labels can clobber them  
            labels[max_overlaps < cfg.TRAIN.RPN_NEGATIVE_OVERLAP] = 0  
  
        # fg label: for each gt, anchor with highest overlap  
        labels[gt_argmax_overlaps] = 1  
  
        # fg label: above threshold IOU  
        labels[max_overlaps >= cfg.TRAIN.RPN_POSITIVE_OVERLAP] = 1  
  
        if cfg.TRAIN.RPN_CLOBBER_POSITIVES:  
            # assign bg labels last so that negative labels can clobber positives  
            labels[max_overlaps < cfg.TRAIN.RPN_NEGATIVE_OVERLAP] = 0  
  
        # subsample positive labels if we have too many  
        num_fg = int(cfg.TRAIN.RPN_FG_FRACTION * cfg.TRAIN.RPN_BATCHSIZE)  
        fg_inds = np.where(labels == 1)[0]  
        if len(fg_inds) > num_fg:  
            disable_inds = npr.choice(  
                fg_inds, size=(len(fg_inds) - num_fg), replace=False)  
            labels[disable_inds] = -1  
  
        # subsample negative labels if we have too many  
        num_bg = cfg.TRAIN.RPN_BATCHSIZE - np.sum(labels == 1)  
        bg_inds = np.where(labels == 0)[0]  
        if len(bg_inds) > num_bg:  
            disable_inds = npr.choice(  
                bg_inds, size=(len(bg_inds) - num_bg), replace=False)  
            labels[disable_inds] = -1  
            #print "was %s inds, disabling %s, now %s inds" % (  
                #len(bg_inds), len(disable_inds), np.sum(labels == 0))  
  
        bbox_targets = np.zeros((len(inds_inside), 4), dtype=np.float32)  
        bbox_targets = _compute_targets(anchors, gt_boxes[argmax_overlaps, :])  
  
        bbox_inside_weights = np.zeros((len(inds_inside), 4), dtype=np.float32)  
        bbox_inside_weights[labels == 1, :] = np.array(cfg.TRAIN.RPN_BBOX_INSIDE_WEIGHTS)  
  
        bbox_outside_weights = np.zeros((len(inds_inside), 4), dtype=np.float32)  
        if cfg.TRAIN.RPN_POSITIVE_WEIGHT < 0:  
            # uniform weighting of examples (given non-uniform sampling)  
            num_examples = np.sum(labels >= 0)  
            positive_weights = np.ones((1, 4)) * 1.0 / num_examples  
            negative_weights = np.ones((1, 4)) * 1.0 / num_examples  
        else:  
            assert ((cfg.TRAIN.RPN_POSITIVE_WEIGHT > 0) &  
                    (cfg.TRAIN.RPN_POSITIVE_WEIGHT < 1))  
            positive_weights = (cfg.TRAIN.RPN_POSITIVE_WEIGHT /  
                                np.sum(labels == 1))  
            negative_weights = ((1.0 - cfg.TRAIN.RPN_POSITIVE_WEIGHT) /  
                                np.sum(labels == 0))  
        bbox_outside_weights[labels == 1, :] = positive_weights  
        bbox_outside_weights[labels == 0, :] = negative_weights  
  
        if DEBUG:  
            self._sums += bbox_targets[labels == 1, :].sum(axis=0)  
            self._squared_sums += (bbox_targets[labels == 1, :] ** 2).sum(axis=0)  
            self._counts += np.sum(labels == 1)  
            means = self._sums / self._counts  
            stds = np.sqrt(self._squared_sums / self._counts - means ** 2)  
            print 'means:'  
            print means  
            print 'stdevs:'  
            print stds  
  
        # map up to original set of anchors  
        labels = _unmap(labels, total_anchors, inds_inside, fill=-1)  
        bbox_targets = _unmap(bbox_targets, total_anchors, inds_inside, fill=0)  
        bbox_inside_weights = _unmap(bbox_inside_weights, total_anchors, inds_inside, fill=0)  
        bbox_outside_weights = _unmap(bbox_outside_weights, total_anchors, inds_inside, fill=0)  
  
        if DEBUG:  
            print 'rpn: max max_overlap', np.max(max_overlaps)  
            print 'rpn: num_positive', np.sum(labels == 1)  
            print 'rpn: num_negative', np.sum(labels == 0)  
            self._fg_sum += np.sum(labels == 1)  
            self._bg_sum += np.sum(labels == 0)  
            self._count += 1  
            print 'rpn: num_positive avg', self._fg_sum / self._count  
            print 'rpn: num_negative avg', self._bg_sum / self._count  
  
        # labels  
        labels = labels.reshape((1, height, width, A)).transpose(0, 3, 1, 2)  
        labels = labels.reshape((1, 1, A * height, width))  
        top[0].reshape(*labels.shape)  
        top[0].data[...] = labels  
  
        # bbox_targets  
        bbox_targets = bbox_targets \
            .reshape((1, height, width, A * 4)).transpose(0, 3, 1, 2)  
        top[1].reshape(*bbox_targets.shape)  
        top[1].data[...] = bbox_targets  
  
        # bbox_inside_weights  
        bbox_inside_weights = bbox_inside_weights \
            .reshape((1, height, width, A * 4)).transpose(0, 3, 1, 2)  
        assert bbox_inside_weights.shape[2] == height  
        assert bbox_inside_weights.shape[3] == width  
        top[2].reshape(*bbox_inside_weights.shape)  
        top[2].data[...] = bbox_inside_weights  
  
        # bbox_outside_weights  
        bbox_outside_weights = bbox_outside_weights \
            .reshape((1, height, width, A * 4)).transpose(0, 3, 1, 2)  
        assert bbox_outside_weights.shape[2] == height  
        assert bbox_outside_weights.shape[3] == width  
        top[3].reshape(*bbox_outside_weights.shape)  
        top[3].data[...] = bbox_outside_weights  
  
    def backward(self, top, propagate_down, bottom):  
        """This layer does not propagate gradients."""  
        pass  
  
    def reshape(self, bottom, top):  
        """Reshaping happens during the call to forward."""  
        pass  
  
  
def _unmap(data, count, inds, fill=0):  
    """ Unmap a subset of item (data) back to the original set of items (of 
    size count) """  
    if len(data.shape) == 1:  
        ret = np.empty((count, ), dtype=np.float32)  
        ret.fill(fill)  
        ret[inds] = data  
    else:  
        ret = np.empty((count, ) + data.shape[1:], dtype=np.float32)  
        ret.fill(fill)  
        ret[inds, :] = data  
    return ret  
  
  
def _compute_targets(ex_rois, gt_rois):  
    """Compute bounding-box regression targets for an image."""  
  
    assert ex_rois.shape[0] == gt_rois.shape[0]  
    assert ex_rois.shape[1] == 4  
    assert gt_rois.shape[1] == 5  
  
    return bbox_transform(ex_rois, gt_rois[:, :4]).astype(np.float32, copy=False)  

In [4]:
height=50;
width=50;
_feat_stride=16 
shift_x = np.arange(0, width) * _feat_stride
shift_y = np.arange(0, height) * _feat_stride
shift_x, shift_y = np.meshgrid(shift_x, shift_y)
shifts = np.vstack((shift_x.ravel(), shift_y.ravel(),
                    shift_x.ravel(), shift_y.ravel())).transpose()

In [6]:
A = _num_anchors = 9
K = shifts.shape[0]
_anchors = generate_anchors()
all_anchors = (_anchors.reshape((1, A, 4)) +  
               shifts.reshape((1, K, 4)).transpose((1, 0, 2)))  
all_anchors = all_anchors.reshape((K * A, 4))  
total_anchors = int(K * A)

In [9]:
all_anchors.shape

(22500, 4)

In [18]:
(all_anchors[:,0]>=0)

array([False, False, False, ...,  True,  True,  True], dtype=bool)

In [17]:
(all_anchors[:,0]>=0).shape

(22500,)

In [25]:
np.where((all_anchors[:,0]>=0)&(all_anchors[:,1]>=0)&(all_anchors[:,2]<800)&(all_anchors[:,3]<800))

(array([ 1404,  1413,  1422, ..., 21069, 21078, 21087]),)

In [27]:
inds_inside = np.where((all_anchors[:,0]>=0)&(all_anchors[:,1]>=0)&(all_anchors[:,2]<800)&(all_anchors[:,3]<800))[0]
inds_inside.shape

(9132,)

In [28]:
anchors = all_anchors[inds_inside,:]

In [29]:
anchors

array([[  12.,    8.,  195.,  103.],
       [  28.,    8.,  211.,  103.],
       [  44.,    8.,  227.,  103.],
       ..., 
       [ 572.,  696.,  755.,  791.],
       [ 588.,  696.,  771.,  791.],
       [ 604.,  696.,  787.,  791.]])

In [35]:
labels = np.empty((len(inds_inside),),dtype=np.float32)
labels.fill(-1)

In [36]:
labels.shape

(9132,)

In [37]:
labels

array([-1., -1., -1., ..., -1., -1., -1.], dtype=float32)

In [48]:
gt_boxes = anchors[[1,100,1000,5000],:]+50
gt_boxes

array([[  78.,   58.,  261.,  153.],
       [ 542.,   74.,  725.,  169.],
       [ 366.,  130.,  453.,  305.],
       [ 426.,  410.,  553.,  537.]])

In [93]:
#GT boxes (x1,y1,x2,y2,label)
gt_boxes =np.array([[  78.,   58.,  261.,  153.,1],
       [ 542.,   74.,  725.,  169.,2],
       [ 366.,  130.,  453.,  305.,3],
       [ 426.,  410.,  553.,  537.,4]])

In [94]:
np.ascontiguousarray(gt_boxes,dtype=np.float)

array([[  78.,   58.,  261.,  153.,    1.],
       [ 542.,   74.,  725.,  169.,    2.],
       [ 366.,  130.,  453.,  305.,    3.],
       [ 426.,  410.,  553.,  537.,    4.]])

In [95]:
overlaps = bbox_overlaps(anchors,gt_boxes)

In [96]:
overlaps

array([[ 0.18153846,  0.        ,  0.        ,  0.        ],
       [ 0.21135647,  0.        ,  0.        ,  0.        ],
       [ 0.24271845,  0.        ,  0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ]])

In [97]:
overlaps.shape

(9132, 4)

In [98]:
argmax_overlaps = overlaps.argmax(axis=1)

In [99]:
argmax_overlaps.shape

(9132,)

In [100]:
max_overlaps = overlaps[np.arange(len(inds_inside)),argmax_overlaps]
max_overlaps

array([ 0.18153846,  0.21135647,  0.24271845, ...,  0.        ,
        0.        ,  0.        ])

In [101]:
max_overlaps.shape

(9132,)

In [102]:
gt_argmax_overlaps = overlaps.argmax(axis=0)

In [103]:
gt_argmax_overlaps

array([ 259,  526, 1680, 5805])

In [104]:
gt_max_overlaps = overlaps[gt_argmax_overlaps,
                          np.arange(overlaps.shape[1])]

In [105]:
gt_max_overlaps

array([ 0.93896817,  0.93896817,  0.93454909,  0.93985318])

In [106]:
overlaps==gt_max_overlaps

array([[False, False, False, False],
       [False, False, False, False],
       [False, False, False, False],
       ..., 
       [False, False, False, False],
       [False, False, False, False],
       [False, False, False, False]], dtype=bool)

In [107]:
(overlaps==gt_max_overlaps).shape

(9132, 4)

In [108]:
a = np.array([[1,2,3],
              [4,5,6]])
b = np.array([4,2,6])

In [109]:
a==b

array([[False,  True, False],
       [ True, False,  True]], dtype=bool)

In [110]:
a = np.array([[1,2,3],
              [4,5,6]])
b = np.array([[2],[6]])

In [111]:
a==b

array([[False,  True, False],
       [False, False,  True]], dtype=bool)

In [112]:
np.where(overlaps==gt_max_overlaps)[0]

array([ 259,  526, 1680, 5805])

In [113]:
gt_argmax_overlap = np.where(overlaps == gt_max_overlaps)[0]

In [114]:
        if not cfg.TRAIN.RPN_CLOBBER_POSITIVES:  
            # assign bg labels first so that positive labels can clobber them  
            labels[max_overlaps < cfg.TRAIN.RPN_NEGATIVE_OVERLAP] = 0  
  
        # fg label: for each gt, anchor with highest overlap  
        labels[gt_argmax_overlaps] = 1  
  
        # fg label: above threshold IOU  
        labels[max_overlaps >= cfg.TRAIN.RPN_POSITIVE_OVERLAP] = 1  
  
        if cfg.TRAIN.RPN_CLOBBER_POSITIVES:  
            # assign bg labels last so that negative labels can clobber positives  
            labels[max_overlaps < cfg.TRAIN.RPN_NEGATIVE_OVERLAP] = 0  
  
        # subsample positive labels if we have too many  
        num_fg = int(cfg.TRAIN.RPN_FG_FRACTION * cfg.TRAIN.RPN_BATCHSIZE)  
        fg_inds = np.where(labels == 1)[0]  
        if len(fg_inds) > num_fg:  
            disable_inds = npr.choice(  
                fg_inds, size=(len(fg_inds) - num_fg), replace=False)  
            labels[disable_inds] = -1  
  
        # subsample negative labels if we have too many  
        num_bg = cfg.TRAIN.RPN_BATCHSIZE - np.sum(labels == 1)  
        bg_inds = np.where(labels == 0)[0]  
        if len(bg_inds) > num_bg:  
            disable_inds = npr.choice(  
                bg_inds, size=(len(bg_inds) - num_bg), replace=False)  
            labels[disable_inds] = -1  
            #print "was %s inds, disabling %s, now %s inds" % (  
                #len(bg_inds), len(disable_inds), np.sum(labels == 0))  

In [115]:
        bbox_targets = np.zeros((len(inds_inside), 4), dtype=np.float32)  
        bbox_targets = _compute_targets(anchors, gt_boxes[argmax_overlaps, :])  

In [116]:
argmax_overlaps

array([0, 0, 0, ..., 0, 0, 0])

In [117]:
gt_boxes

array([[  78.,   58.,  261.,  153.,    1.],
       [ 542.,   74.,  725.,  169.,    2.],
       [ 366.,  130.,  453.,  305.,    3.],
       [ 426.,  410.,  553.,  537.,    4.]])

In [120]:
gt_boxes[argmax_overlaps, :]

array([[  78.,   58.,  261.,  153.,    1.],
       [  78.,   58.,  261.,  153.,    1.],
       [  78.,   58.,  261.,  153.,    1.],
       ..., 
       [  78.,   58.,  261.,  153.,    1.],
       [  78.,   58.,  261.,  153.,    1.],
       [  78.,   58.,  261.,  153.,    1.]])

In [121]:
(gt_boxes[argmax_overlaps, :]).shape

(9132, 5)

In [122]:
anchors.shape

(9132, 4)

In [119]:
bbox_targets.shape

(9132, 4)

In [127]:
print(bbox_targets[100:110,:])

[[ 0.27173913  0.52083331  0.          0.        ]
 [ 0.390625    0.390625    0.3629055  -0.28768209]
 [ 0.18478261  0.52083331  0.          0.        ]
 [ 0.265625    0.390625    0.3629055  -0.28768209]
 [ 0.09782609  0.52083331  0.          0.        ]
 [ 0.140625    0.390625    0.3629055  -0.28768209]
 [ 0.01086957  0.52083331  0.          0.        ]
 [ 0.015625    0.390625    0.3629055  -0.28768209]
 [-0.07608695  0.52083331  0.          0.        ]
 [-0.109375    0.390625    0.3629055  -0.28768209]]


In [123]:
bbox_targets.shape

(9132, 4)

In [124]:
import numpy as np

def bbox_transform(ex_rois, gt_rois):
    ex_widths = ex_rois[:, 2] - ex_rois[:, 0] + 1.0#width
    ex_heights = ex_rois[:, 3] - ex_rois[:, 1] + 1.0#height
    ex_ctr_x = ex_rois[:, 0] + 0.5 * ex_widths#x_center
    ex_ctr_y = ex_rois[:, 1] + 0.5 * ex_heights#y_center

    gt_widths = gt_rois[:, 2] - gt_rois[:, 0] + 1.0#gt_width
    gt_heights = gt_rois[:, 3] - gt_rois[:, 1] + 1.0#gt_height
    gt_ctr_x = gt_rois[:, 0] + 0.5 * gt_widths#gt_x_cebter
    gt_ctr_y = gt_rois[:, 1] + 0.5 * gt_heights#gt_y_center

    targets_dx = (gt_ctr_x - ex_ctr_x) / ex_widths#dx=(gt_cx-anchor_cx)/anchor_w
    targets_dy = (gt_ctr_y - ex_ctr_y) / ex_heights#dy=(gt_cy-anchor_cy)/anchor_h
    targets_dw = np.log(gt_widths / ex_widths)#log(gt_w/anchor_w)
    targets_dh = np.log(gt_heights / ex_heights)#log(gt_h/anchor_h)

    targets = np.vstack(
        (targets_dx, targets_dy, targets_dw, targets_dh)).transpose()
    return targets

def bbox_transform_inv(boxes, deltas):#上一个函数的逆运算，由delta值和对应的anchor计算出box坐标
    if boxes.shape[0] == 0:
        return np.zeros((0, deltas.shape[1]), dtype=deltas.dtype)

    boxes = boxes.astype(deltas.dtype, copy=False)

    widths = boxes[:, 2] - boxes[:, 0] + 1.0#anchor_w
    heights = boxes[:, 3] - boxes[:, 1] + 1.0#anchor_h
    ctr_x = boxes[:, 0] + 0.5 * widths#anchor_cx
    ctr_y = boxes[:, 1] + 0.5 * heights#anchor_cy

    dx = deltas[:, 0::4]
    dy = deltas[:, 1::4]
    dw = deltas[:, 2::4]
    dh = deltas[:, 3::4]

    pred_ctr_x = dx * widths[:, np.newaxis] + ctr_x[:, np.newaxis]#dx*anchor_w+anchor_cx
    pred_ctr_y = dy * heights[:, np.newaxis] + ctr_y[:, np.newaxis]#dy*abchor_h+anchor_cy
    pred_w = np.exp(dw) * widths[:, np.newaxis]#exp(dw)*anchor_w
    pred_h = np.exp(dh) * heights[:, np.newaxis]#exp(dh)*anchor_h

    pred_boxes = np.zeros(deltas.shape, dtype=deltas.dtype)#下面是由预测框的中心和长宽还原成左上右下坐标
    # x1
    pred_boxes[:, 0::4] = pred_ctr_x - 0.5 * pred_w
    # y1
    pred_boxes[:, 1::4] = pred_ctr_y - 0.5 * pred_h
    # x2
    pred_boxes[:, 2::4] = pred_ctr_x + 0.5 * pred_w
    # y2
    pred_boxes[:, 3::4] = pred_ctr_y + 0.5 * pred_h

    return pred_boxes

def clip_boxes(boxes, im_shape):
    """
    Clip boxes to image boundaries.
    """

    # x1 >= 0
    boxes[:, 0::4] = np.maximum(np.minimum(boxes[:, 0::4], im_shape[1] - 1), 0)
    # y1 >= 0
    boxes[:, 1::4] = np.maximum(np.minimum(boxes[:, 1::4], im_shape[0] - 1), 0)
    # x2 < im_shape[1]
    boxes[:, 2::4] = np.maximum(np.minimum(boxes[:, 2::4], im_shape[1] - 1), 0)
    # y2 < im_shape[0]
    boxes[:, 3::4] = np.maximum(np.minimum(boxes[:, 3::4], im_shape[0] - 1), 0)
    return boxes

In [128]:
ex_rois = anchors
gt_rois = gt_boxes[argmax_overlaps,:4]

In [130]:
    ex_widths = ex_rois[:, 2] - ex_rois[:, 0] + 1.0#width
    ex_heights = ex_rois[:, 3] - ex_rois[:, 1] + 1.0#height
    ex_ctr_x = ex_rois[:, 0] + 0.5 * ex_widths#x_center
    ex_ctr_y = ex_rois[:, 1] + 0.5 * ex_heights#y_center

    gt_widths = gt_rois[:, 2] - gt_rois[:, 0] + 1.0#gt_width
    gt_heights = gt_rois[:, 3] - gt_rois[:, 1] + 1.0#gt_height
    gt_ctr_x = gt_rois[:, 0] + 0.5 * gt_widths#gt_x_cebter
    gt_ctr_y = gt_rois[:, 1] + 0.5 * gt_heights#gt_y_center

    targets_dx = (gt_ctr_x - ex_ctr_x) / ex_widths#dx=(gt_cx-anchor_cx)/anchor_w
    targets_dy = (gt_ctr_y - ex_ctr_y) / ex_heights#dy=(gt_cy-anchor_cy)/anchor_h
    targets_dw = np.log(gt_widths / ex_widths)#log(gt_w/anchor_w)
    targets_dh = np.log(gt_heights / ex_heights)#log(gt_h/anchor_h)

    targets = np.vstack(
        (targets_dx, targets_dy, targets_dw, targets_dh)).transpose()

In [144]:
ex_rois[100:110,:],ex_widths[100:110],ex_heights[100:110]

(array([[ 492.,   24.,  675.,  119.],
        [ 520.,    8.,  647.,  135.],
        [ 508.,   24.,  691.,  119.],
        [ 536.,    8.,  663.,  135.],
        [ 524.,   24.,  707.,  119.],
        [ 552.,    8.,  679.,  135.],
        [ 540.,   24.,  723.,  119.],
        [ 568.,    8.,  695.,  135.],
        [ 556.,   24.,  739.,  119.],
        [ 584.,    8.,  711.,  135.]]),
 array([ 184.,  128.,  184.,  128.,  184.,  128.,  184.,  128.,  184.,  128.]),
 array([  96.,  128.,   96.,  128.,   96.,  128.,   96.,  128.,   96.,  128.]))

In [143]:
gt_rois[100:110,:],gt_widths[100:110],gt_heights[100:110]

(array([[ 542.,   74.,  725.,  169.],
        [ 542.,   74.,  725.,  169.],
        [ 542.,   74.,  725.,  169.],
        [ 542.,   74.,  725.,  169.],
        [ 542.,   74.,  725.,  169.],
        [ 542.,   74.,  725.,  169.],
        [ 542.,   74.,  725.,  169.],
        [ 542.,   74.,  725.,  169.],
        [ 542.,   74.,  725.,  169.],
        [ 542.,   74.,  725.,  169.]]),
 array([ 184.,  184.,  184.,  184.,  184.,  184.,  184.,  184.,  184.,  184.]),
 array([ 96.,  96.,  96.,  96.,  96.,  96.,  96.,  96.,  96.,  96.]))

In [146]:
gt_widths[100:110]/ex_widths[100:110]

array([ 1.    ,  1.4375,  1.    ,  1.4375,  1.    ,  1.4375,  1.    ,
        1.4375,  1.    ,  1.4375])

In [147]:
np.log(gt_widths[100:110]/ex_widths[100:110])

array([ 0.        ,  0.36290549,  0.        ,  0.36290549,  0.        ,
        0.36290549,  0.        ,  0.36290549,  0.        ,  0.36290549])

In [132]:
targets_dx.shape

(9132,)

In [136]:
targets[100:110]

array([[ 0.27173913,  0.52083333,  0.        ,  0.        ],
       [ 0.390625  ,  0.390625  ,  0.36290549, -0.28768207],
       [ 0.18478261,  0.52083333,  0.        ,  0.        ],
       [ 0.265625  ,  0.390625  ,  0.36290549, -0.28768207],
       [ 0.09782609,  0.52083333,  0.        ,  0.        ],
       [ 0.140625  ,  0.390625  ,  0.36290549, -0.28768207],
       [ 0.01086957,  0.52083333,  0.        ,  0.        ],
       [ 0.015625  ,  0.390625  ,  0.36290549, -0.28768207],
       [-0.07608696,  0.52083333,  0.        ,  0.        ],
       [-0.109375  ,  0.390625  ,  0.36290549, -0.28768207]])

In [148]:
np.vstack(([1,2],[3,4]))

array([[1, 2],
       [3, 4]])

In [149]:
np.vstack(([1,2],[3,4])).transpose()

array([[1, 3],
       [2, 4]])

In [152]:
labels.shape

(9132,)

In [153]:
total_anchors

22500

In [155]:
inds_inside.shape

(9132,)

In [156]:
def _unmap(data, count, inds, fill=0):  
    """ Unmap a subset of item (data) back to the original set of items (of 
    size count) """  
    if len(data.shape) == 1:  
        ret = np.empty((count, ), dtype=np.float32)  
        ret.fill(fill)  
        ret[inds] = data  
    else:  
        ret = np.empty((count, ) + data.shape[1:], dtype=np.float32)  
        ret.fill(fill)  
        ret[inds, :] = data  
    return ret 

In [157]:
labels = _unmap(labels, total_anchors, inds_inside, fill=-1)

In [159]:
labels.shape

(22500,)

In [163]:
(total_anchors,)+labels.shape[1:]

(22500,)

In [164]:
(total_anchors,)+bbox_targets.shape[1:]

(22500, 4)

In [165]:
bbox_targets.shape[1:]

(4,)

In [166]:
(22500,)+(4,)

(22500, 4)